In [1]:
%matplotlib widget
import bmcs_utils.api as bu
from bmcs_cross_section.api import ReinfLayer
from bmcs_shear.shear_crack.crack_tip_orientation import CrackStateAnimator
ce = CrackStateAnimator()
ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
x_00=250
ce.sz_cp.trait_set(x_00=x_00, n_m = 15)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=300, B=200, L=500)
ce.sz_cp.add_x_tip_an([x_00-10, 50]) 
ce.sz_cp.add_x_tip_an([x_00-30, 100]) 

bl1 = ReinfLayer(name='layer1', z=44, A=ce.sz_bd.smm.A_f, matmod='steel')
bl1.matmod_.trait_set(E=ce.sz_bd.smm.E_f, f_t=ce.sz_bd.smm.sig_y)
ce.sz_bd.cross_section_layout.add_layer(bl1)
#ce.sz_cp.sz_ctr.w = 5

In [2]:
# %matplotlib widget
# import bmcs_utils.api as bu
# from bmcs_shear.shear_crack.crack_path import CrackPathAnimator
# cp = CrackPathAnimator()
# #ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
# x_00=250
# cp.trait_set(x_00=x_00, n_m = 15)
# cp.sz_bd.Rectangle = True
# cp.sz_bd.trait_set(H=300, B=200, L=500)
# #cp.sz_cp.add_x_tip_an([x_00-10, 50]) 
# #cp.sz_cp.add_x_tip_an([x_00-30, 100]) 
# #ce.sz_cp.sz_ctr.w = 5

In [3]:
ce.interact()

<lambdifygenerated-26>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


In [4]:
%%capture
import numpy as np
import matplotlib.pylab as plt
from matplotlib import animation, rc

ce = CrackStateAnimator()
ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
x_00=250
ce.sz_cp.trait_set(x_00=x_00, n_m = 10)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=300, B=200, L=500)
ce.sz_cp.add_x_tip_an([x_00-10, 50]) 
ce.sz_cp.add_x_tip_an([x_00-30, 100]) 
#ce.sz_cp.sz_ctr.w = 30

fig = plt.figure(figsize=(12,5))
fig.canvas.header_visible = False

psi_range = np.linspace(0, np.pi/4, 15)
psi_range = np.hstack([psi_range, psi_range[-1:0:-1]])
ce.sz_cp.sz_ctr.x_rot_1k = 200
x_rot_1k = ce.sz_cp.sz_ctr.x_rot_1k

def plot_stress_state(i):
    ce.psi_slider = psi_range[i]
    fig.clf()
    axes = ce.sz_stress_profile.subplots(fig)
    ce.sz_stress_profile.update_plot(axes)
    ax_u_0, ax_w_0, ax_S_0, ax_F_0, ax_u_1, ax_w_1, ax_S_1, ax_F_1 = axes
    x_factor = 0.11
    ax_u_0.set_xlim(xmin=-0.5*x_factor, xmax=1*x_factor)
    x_factor = 0.05
    ax_u_1.set_xlim(xmin=-0.5*x_factor, xmax=1*x_factor)
    x_factor = 0.11
    ax_w_0.set_xlim(xmin=-0.6*x_factor, xmax=1*x_factor)
    x_factor = 0.06
    ax_w_1.set_xlim(xmin=-0.6*x_factor, xmax=1*x_factor)
    x_factor = 1000
    ax_S_0.set_xlim(xmin=-3*x_factor, xmax=1*x_factor)
    x_factor = 350
    ax_S_1.set_xlim(xmin=-3*x_factor, xmax=1*x_factor)
    x_factor = 1040
    ax_F_0.set_xlim(xmin=-1.7*x_factor, xmax=1*x_factor)
    x_factor = 300
    ax_F_1.set_xlim(xmin=-1.7*x_factor, xmax=1*x_factor)
    
def plot_deformed_state(i):
    ce.psi_slider = psi_range[i]
    ce.x_rot_1k_slider = x_rot_1k
    fig.clf()
    axes = ce.sz_stress_profile.ds.subplots(fig)
    ce.sz_stress_profile.ds.update_plot(axes)
    axes.set_ylim(ymin=-10, ymax=310)
    axes.set_xlim(xmin = 295, xmax=296)
    

In [5]:
def html5_stress_state_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_stress_state,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()

def html5_deformed_state_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_deformed_state,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()

In [6]:
import os.path
home_dir = os.path.expanduser('~')
anim_dir = os.path.join(home_dir, 'generated_animation')
if not os.path.exists(anim_dir):
    os.makedirs(anim_dir)

In [7]:
%%capture
deformed_anim_file = os.path.join(anim_dir, 'deformed_state_animation.html')
with open(deformed_anim_file,'w') as html_video_file:
    html_video_file.write(html5_deformed_state_video())

In [8]:
from IPython.display import HTML
html_video_file = open(deformed_anim_file,'r')
HTML(html_video_file.read())

In [9]:
%%capture
stress_anim_file = os.path.join(anim_dir, 'stress_state_animation.html')
with open(stress_anim_file,'w') as html_video_file:
    html_video_file.write(html5_stress_state_video())

In [10]:
from IPython.display import HTML
html_video_file = open(stress_anim_file,'r')
HTML(html_video_file.read())

In [8]:
fig, ax = plt.subplots(1,1)
ds = ce.crack_tip_shear_stress.sz_stress_profile.ds
ds.plot_sz1(ax)
#ds.sz_cp.plot_sz0(ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig, ax = plt.subplots(1,1)
ds = ce.plot_geo
ds

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'CrackStateAnimator' object has no attribute 'plot_geo'

In [11]:
rect = 0, 0, 0.25, 1
fig = plt.figure()
ax1 = fig.add_axes((0,0,0.25,1), label='label1')
ax2 = fig.add_axes((0.25,0,0.25,1), label='label1', sharey=ax1)
ax3 = fig.add_axes((0.5,0,0.25,1), frameon=True, facecolor='g')
ax4 = fig.add_axes((0.75,0,0.25,1), polar=True)
#ax = fig.add_axes(rect, projection='polar')
#fig.delaxes(ax)
#fig.add_axes(ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …